In [20]:
from google.colab import auth
auth.authenticate_user()
import Embed_Cluster as ec ##Custom library
import gspread
import pandas as pd
from google.auth import default
creds, _ = default()
# Import customer response sheet
gc = gspread.authorize(creds)
worksheet = gc.open('Athens Trip Maker (Responses)').sheet1

# get_all_values gives a list of rows and convert to pandas df
rows = worksheet.get_all_values()
customer_df = pd.DataFrame.from_records(rows)

**Remember to add the below files to directory to run all the routines**

In [21]:
df_hotels_c = pd.read_csv('df_hotels_c.csv')
df_attractions_c = pd.read_csv('df_attractions_c.csv')
df_restaurants_c = pd.read_csv('df_restaurants_c.csv')

The above dataframe is pulled from our customer survey which contains the following columns:

**0** Timestamp <br>
**1** Username <br>
**2** Who's traveling(partner, family, solo, friends) <br>
**3** Number of adults <br>
**4** Number of children <br>
**5** Length of trip(days) <br>
**6:16**  Hotel Clusters(Yes/Maybe/No) <br>
**17:30** Attraction/Activity Clusters(Yes/Maybe/No) <br>
**31:45** Restaurant Clusters(Yes/Maybe/No) <br>

Below we write a function that will parse the above data into a format that is more easily readable e.g. transform into vectors that contain the cluster numbers corresponding to each component

In [22]:
def parse_response(response):
  ##### For now we only include clusters that have Yes response, but can also add the Maybes later down the road, or make a more complex algorithm that
  ##### incorporates them proportionately less than 'Yes'
  hotels = []
  attractions = []
  restaurants = []
  for i in range(6,17):
    if response[i] == 'Yes':
      hotels.append(i-6)
  for i in range(17,31):
    if response[i] == 'Yes':
      attractions.append(i-17)
  for i in range(31,46):
    if response[i] == 'Yes':
      restaurants.append(i-31)
  return (hotels, attractions, restaurants)

Below we introduce a function where we will put all our customer input together and create a prompt to pass to the LLM that will synthesize our itinerary. We should also return a raw list of the components that could be useful later on.

In [5]:
def create_prompt(username, customer_df):
    response = customer_df[customer_df[1]==username].values.tolist()
    response = response[0]
    trip_style = response[2] #partner, family, solo, friends
    num_adults = response[3]
    num_children = response[4]
    num_nights = int(response[5])
    misc = response[46]
    ####This part of the prompt is the list of hotels, attractions, and restaurants
    components = parse_response(response)
    hotel_components =  ec.get_components(num_nights,components[0], df_hotels_c, 3)
    attraction_components =  ec.get_components(num_nights,components[1], df_attractions_c, 4)
    restaurant_components =  ec.get_components(num_nights,components[2], df_restaurants_c, 5)
    components = 'HOTELS: ' +  ", ".join(hotel_components) + '. '\
    + 'RESTAURANTS: ' +  ", ".join(restaurant_components) + '. '\
    + 'ATTRACTIONS: ' +  ", ".join(attraction_components) + '. '

    ####We now build the full prompt that we will give to the LLM

    prompt = 'Hi, I would like to plan a unique and special itinerary for ' + trip_style + ' to Athens, Greece for ' + str(num_nights) + ' nights. ' + \
    'There will be ' + num_adults + ' adults' + ' and ' + num_children + ' children ' + 'including myself. ' + \
    'Can you please put together a detailed itinerary day-by-day that includes accommodations, meals and any other helpful suggestions? Here are some additional details as well - please pay special attention to these: ' + \
    misc + '. Do not forget to minimise travel distance, and explain transportation in detail too. If you mention to take a guided tour, you should suggest at least one specific guided tour company. ' + \
    'I have narrowed down some of my favorite hotels, restaurants and activities below:' + components + 'Please make sure to give me 2 different hotel options to choose from. Make sure to' + \
    ' state which restaurants or locations where we will eat. There must be a specific suggested restaurant for every meal. Give approximate travel time to/from each activity.'



    return (prompt, hotel_components, attraction_components, restaurant_components)

Here we now import our LLM which in this case is Google's Gemini model. These contain the following parameters:

**Top-p Sampling**

By using Top-p sampling, you allow for a more controlled and diverse generation process. It avoids overly repetitive or deterministic outputs that can arise from always selecting the most likely token. Instead, the model has a chance to select from a wider range of options within the nucleus, which promotes more varied and creative text generation.

The value of “p” determines the diversity of the generated output. Higher values of “p” result in more randomness and diversity, as a larger number of tokens are considered for sampling. Lower values of “p” lead to more focused and deterministic output, as the model focuses on the most likely tokens.

Top-P changes how the model selects tokens for output. Tokens are selected from the most (see top-K) to least probable until the sum of their probabilities equals the top-P value. For example, if tokens A, B, and C have a probability of 0.6, 0.3, 0.1 and the top-P value is 0.9, then the model will select either A or B as the next token by using temperature and excludes C as a candidate.
Specify a lower value for less random responses and a higher value for more random responses. The default value of top-P is 1.0.

**Top-k Sampling**

Top-K changes how the model selects tokens for output. A top-K of 1 means the next selected token is the most probable among all tokens in the model's vocabulary (also called greedy decoding), while a top-K of 3 means that the next token is selected from among the three most probable tokens by using temperature.
For each token selection step, the top-K tokens with the highest probabilities are sampled. Then tokens are further filtered based on top-P with the final token selected using temperature sampling.

Specify a lower value for less random responses and a higher value for more random responses. The default value of top-K is 32.

**Temperature**

A higher temperature value (> 1.0) increases the randomness and diversity of the generated output. It makes the model assign more equal probabilities to a wider range of tokens, allowing for more exploration and creative variations in the generated text. This can result in more unexpected and diverse output, but it may also introduce more noise or less coherent responses.

On the other hand, a lower temperature value (< 1.0) reduces the randomness and encourages the model to focus on the most probable tokens. It makes the distribution peakier, with higher probabilities assigned to the most likely tokens. This can lead to more deterministic and conservative output, with less variation and more coherent responses.


In [6]:
import vertexai
vertexai.init(project='travelgen-410711', location="us-central1")

import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.generative_models import GenerativeModel

# Load Gemini Pro
gemini_pro_model = GenerativeModel("gemini-pro")

# Load Gemini Pro Vision
gemini_pro_vision_model = GenerativeModel("gemini-pro-vision")

In [7]:
def generate(prompt, temperature, top_p, top_k):
  #model = GenerativeModel("gemini-pro-vision")
  model = GenerativeModel("gemini-pro")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k
    },
  stream=True,
  )

  text = ''
  for i in responses:
    text+=i.text
  return text

In [23]:
## Extract the prompt, and raw components.
prompt_output = create_prompt('bubonbek1601@gmail.com', customer_df)
prompt = prompt_output[0]

In [26]:
prompt

"Hi, I would like to plan a unique and special itinerary for Me and friends to Athens, Greece for 7 nights. There will be 5 adults and 0 children including myself. Can you please put together a detailed itinerary day-by-day that includes accommodations, meals and any other helpful suggestions? Here are some additional details as well - please pay special attention to these: I really like food. And a cozy bed which i can collapse into. And a cocktail.. Do not forget to minimise travel distance, and explain transportation in detail too. If you mention to take a guided tour, you should suggest at least one specific guided tour company. I have narrowed down some of my favorite hotels, restaurants and activities below:HOTELS: Hilton Athens, The Athens Gate Hotel, Hotel Grande Bretagne, A Luxury Collection Hotel, O&B Athens Boutique Hotel, Grand Hyatt Athens, Plaka Hotel, Electra Palace Athens, InnAthens, Wyndham Grand Athens, Attalos Hotel, Herodion Hotel, Coco-Mat Hotel Athens, Crowne Plaz

In [ ]:
responses = generate(prompt, 0.0, 1.0, 25)
display(Markdown(responses))

**Day 1**

* **Morning:** Arrive at Athens International Airport (ATH) and take the metro to Syntagma Square (approx. 45 minutes).
* **Hotel:** Check in to The Athens Gate Hotel (luxury option) or Plaka Hotel (mid-range option).
* **Lunch:** Psomi & Alati (traditional Greek cuisine)
* **Afternoon:** Visit the Acropolis Museum (approx. 1 hour).
* **Evening:** Dinner at Lithos (rooftop restaurant with panoramic views)

**Day 2**

* **Morning:** Explore the ancient ruins of the Acropolis (approx. 2 hours).
* **Lunch:** Mani Mani (authentic Peloponnesian cuisine)
* **Afternoon:** Take a guided tour of Plaka with Chat Tours (approx. 2 hours).
* **Evening:** Dinner at The Greco's Project (modern Greek gastronomy)

**Day 3**

* **Morning:** Visit the National Archaeological Museum (approx. 2 hours).
* **Lunch:** Efcharis (traditional Greek tavern)
* **Afternoon:** Take an all-day cruise to the Saronic Islands with Athens Cruises (approx. 8 hours).
* **Evening:** Dinner on board the cruise ship

**Day 4**

* **Morning:** Visit the Benaki Museum - Museum of Greek Culture (approx. 2 hours).
* **Lunch:** All That Jatz (Mediterranean cuisine)
* **Afternoon:** Explore the vibrant Monastiraki neighborhood.
* **Evening:** Farewell dinner at GB Roof Garden (rooftop restaurant with stunning city views)

**Transportation:**

* Metro: Use the Athens Metro to travel between the airport, Syntagma Square, and the Acropolis Museum (approx. 15-20 minutes).
* Walking: Most attractions in the city center are within walking distance of each other.
* Taxi: Taxis are readily available and affordable.

**Additional Tips:**

* Book your hotel and guided tours in advance, especially during peak season.
* Purchase an Athens City Pass for discounted admission to museums and attractions.
* Learn a few basic Greek phrases to enhance your interactions.
* Allow ample time for walking and exploring the city's charming streets.
* Consider visiting during the shoulder season (April-May or September-October) for fewer crowds and more pleasant weather.

We can see above the result given the extensive prompt. While the itinerary created is good, there are still a few aspects which the LLM did not correctly, and others that from user feedback could be incorporated. Initial user comments:

1. How to get from one activity to another and how long it takes is not included
2. Would be nice to include links to the activities*(could incorporate via RAG/API)*
3. Some of the itinerary looks quite ambitious e.g. a lot of activities in one day, the amount of activities that are feasible will depend on the traveler
4. There are some activities that are too general e.g. Enjoy rock concert at local venue. This doesn't state where or how this could work.


To remedy some of these, we will implement a few-shot response as opposed to one-shot. Context is generally provided to the LLM by giving it the previous queries/responses. This moving context window for Google's Gemini gives us 2048 tokens(around 2048 words) so we will keep this mind.

In [24]:
chat = gemini_pro_model.start_chat()
from vertexai.preview.generative_models import ChatSession

def get_chat_response(chat, prompt):
    response = chat.send_message(prompt)
    return response.text




In [25]:
response1 = get_chat_response(chat, prompt)

prompt2 = "Keep the itinerary text and formatting as is, and make sure that method of transportation and how long it takes is given for every meal and activity. Optimize the itinerary to minimize travel distance by grouping activities/restaurants in a similar location. Make sure to include meals for breakfast, lunch and dinner. \
Make sure to mention a specific tour company for any tours. "
response2 = get_chat_response(chat, prompt2)

prompt3 = "Add the approximate costs for each activity and restaurant, keeping the formatting as is and make sure there is are at least 2 contrasting choices of hotel. Make sure the distances between locations are correct. There are some walking distances that are incorrect"
response3 = get_chat_response(chat, prompt3)



In [27]:
prompt4 = 'Please  give me the itinerary again with the following prioritized: I really like food. And a cozy bed which i can collapse into. And a cocktail.'
response4 = get_chat_response(chat, prompt4)

In [ ]:
display(Markdown(response3))

**Day 1**

* **Morning:** Check in to your hotel, **The Artist Athens** (approx. €120/night).
* **Breakfast:** Enjoy breakfast at your hotel's rooftop terrace (approx. €15).
* **Afternoon:** Visit the **Acropolis Museum** (approx. €10 entrance fee; 45 min. walk).
* **Lunch:** Have a light lunch at the Acropolis Museum's café (approx. €10).
* **Dinner:** Have a traditional Greek meal at **Psomi & Alati** near the Acropolis (approx. €20; 15 min. walk).

**Day 2**

* **Morning:** Take a **guided tour of the Acropolis** with **Chat Tours** (approx. €30; 2 hours).
* **Lunch:** Enjoy a post-tour lunch at **Strofi** restaurant near the Acropolis (approx. €15; 5 min. walk).
* **Afternoon:** Stroll through the picturesque neighborhood of **Plaka** (15 min. walk). Visit boutiques and admire the charming architecture.
* **Dinner:** Dine at **Mani Mani** in Plaka for authentic Greek cuisine (approx. €25; 20 min. walk).

**Day 3**

* **Morning:** Visit the **National Archaeological Museum** (approx. €10 entrance fee; 20 min. walk). Discover a vast collection of Greek antiquities, including sculptures, pottery, and jewelry.
* **Lunch:** Have a quick lunch at the museum's café (approx. €10).
* **Afternoon:** Explore the vibrant flea market in **Monastiraki** (15 min. walk). Browse for unique souvenirs and antiques.
* **Dinner:** Enjoy a rooftop meal with panoramic city views at **V1935 Roof Garden** near Syntagma Square (approx. €30; 15 min. walk).

**Day 4**

* **Morning:** Take a day trip to the island of **Hydra** by ferry (approx. €25 round-trip; 1.5 hours by ferry). Explore the car-free streets, visit art galleries, and enjoy the scenic harbor.
* **Lunch:** Have a seafood lunch at a restaurant on Hydra (approx. €20).
* **Afternoon:** Return to Athens in the evening.
* **Dinner:** Have a farewell meal at **Belpaese** (approx. €25; 15 min. walk).

**Day 5**

* **Morning:** Visit the **Museum of Cycladic Art** (approx. €10 entrance fee; 30 min. walk). Admire the extensive collection of ancient Cycladic sculptures and artifacts.
* **Lunch:** Have lunch at the museum's café (approx. €10).
* **Afternoon:** Take an **Athens by Bike** guided tour (approx. €35; 3 hours; book online). Explore the city's highlights from a unique perspective.
* **Dinner:** Try modern Greek cuisine at **Thes "Greek Creative Cuisine"** (approx. €25; 25 min. walk).

**Day 6**

* **Morning:** Check out of your hotel.
* **Breakfast:** Enjoy a final breakfast at your hotel (approx. €15).
* **Afternoon:** Visit the **Benaki Museum** (approx. €10 entrance fee; 20 min. walk). Explore its collections of Greek and Islamic art.
* **Lunch:** Have a light lunch at the museum's café (approx. €10).
* **Evening:** Depart from Athens.

**Alternative Hotel:**

* **Coco-Mat Hotel Athens** (approx. €150/night): An eco-friendly hotel with a rooftop pool and a focus on sustainability, located in Kolonaki.

**Note:** The walking distances and times provided are approximate and may vary depending on your pace and the specific route you take.

In [28]:
display(Markdown(response4))

## Optimized 7-Night Athens Itinerary for Foodies: Prioritizing Food, Cozy Beds, and Cocktails

This revised itinerary prioritizes your love for delicious food, comfortable accommodations, and enjoying a cocktail or two. It ensures you have cozy beds to sink into after a day of exploring and indulging in culinary delights.

### Day 1:

* **Morning:** Arrive at Athens International Airport (ATH). Take the metro line 3 (blue line) directly to Syntagma Square (~40 minutes, €10). Check into your chosen hotel:
    * **Luxury:** Hotel Grande Bretagne, A Luxury Collection Hotel (5-star, centrally located, stunning views, €350+/night, luxurious beds, elegant rooftop bar)
    * **Mid-range:** O&B Athens Boutique Hotel (4-star, stylish, rooftop terrace, €150+/night, comfortable beds, rooftop bar with city views)
* **Lunch:** Enjoy a traditional Greek lunch at **Ta Karamanlidika tou Fani** in Plaka (5-minute walk from Syntagma Square, €20-30 per person, delicious meze platters, friendly atmosphere).
* **Afternoon:** Explore the Acropolis complex, including the Parthenon, Erechtheion, and Temple of Athena Nike. Join a guided tour with **Nectar & Ambrosia** for in-depth insights and skip-the-line access. (~2-hour tour, €70 per person)
* **Evening:** Dine at **Oroscopo**, a Michelin-starred restaurant with innovative Greek cuisine and stunning Acropolis views. Afterward, head to the rooftop bar **A for Athens** for cocktails with panoramic city vistas. (Both within a 5-minute walk of the Acropolis, €50-70 per person for dinner, €15-20 per cocktail)

### Day 2:

* **Morning:** Visit the National Archaeological Museum, home to an extensive collection of ancient Greek artifacts. (~15-minute walk from Plaka or a short metro ride, €10 entrance fee)
* **Lunch:** Have a casual lunch at **Smile Restaurant**, offering traditional Greek cuisine near the museum (~5-minute walk, €15-20 per person, delicious homemade dishes, friendly service).
* **Afternoon:** Stroll through the charming Plaka district, with its narrow streets, souvenir shops, and historical buildings.
* **Evening:** Experience a unique dining experience at **Dinner in the Sky**, enjoying a gourmet meal suspended 50 meters above the ground with breathtaking views. (Location varies, transportation time may vary, €300 per person)
* **Later:** Relax and enjoy a cocktail at the **Brettos Bar** in Monastiraki, renowned for its live music and energetic atmosphere.

### Day 3:

* **Morning:** Take a scenic hike up Mount Lycabettus, offering panoramic views of the city. (~15-minute walk from Syntagma Square or a short taxi ride, €5-10 taxi fare)
* **Lunch:** Have a picnic lunch at the top of Mount Lycabettus, enjoying the breathtaking vistas.
* **Afternoon:** Learn to cook traditional Greek dishes at a hands-on cooking class with **Cookly** in Monastiraki. (~15-minute walk from Mount Lycabettus or a short metro ride, €70 per person)
* **Evening:** Enjoy the fruits of your labor at the cooking class itself, savoring the dishes you prepared.
* **Later:** Experience the vibrant nightlife of Athens at **Brettos Bar** in Monastiraki, renowned for its live music and energetic atmosphere.

### Day 4:

* **Morning:** Visit the Benaki Museum, housing a diverse collection of Greek art and history. (~10-minute walk from Monastiraki or a short metro ride, €10 entrance fee)
* **Lunch:** Have a delightful lunch at **Funky Gourmet** in Neos Kosmos, a Michelin-starred restaurant with creative and playful interpretations of Greek cuisine (~30-minute metro ride from Monastiraki, €50-70 per person, creative dishes, innovative cocktails).
* **Afternoon:** Relax and rejuvenate at a traditional hammam, such as the **Hammam Baths of Athens** in Monastiraki. (~10-minute walk from the Benaki Museum, €40 per person)
* **Evening:** Dine at **Botrini's** in Kifissia, a contemporary fine dining restaurant with a focus on seasonal ingredients. (~40-minute taxi ride from Monastiraki, €100+ per person, exquisite cuisine, sophisticated ambiance).

### Day 5:

* **Morning:** Take a full-day cruise to the Saronic Islands, including Hydra, Poros, and Aegina. Relax on the beaches, explore the picturesque villages, and enjoy swimming in the crystal-clear waters. (~1-hour ferry ride from Piraeus port, €50-70 per person)
* **Lunch:** Enjoy a delicious seafood lunch onboard the cruise.
* **Dinner:** Have a casual dinner at a taverna on one of the islands, savoring the local cuisine and ambiance.

### Day 6:

* **Morning:** Visit the Acropolis Museum, showcasing the archaeological finds from the Acropolis. (~15-minute walk from Plaka or a short metro ride, €10 entrance fee)
* **Lunch:** Have a traditional Greek lunch at **Oroscopo** in Plaka, a Michelin-starred restaurant with innovative Greek cuisine and stunning Acropolis views.
* **Afternoon:** Explore the vibrant Monastiraki flea market, searching for unique souvenirs and hidden treasures.
* **Evening:** Spend a memorable evening at the **Gazarte**, a cultural center with live music, art exhibitions, and a variety of restaurants and bars. (10-minute walk from Monastiraki)

### Day 7:

* **Morning:** Enjoy a leisurely breakfast at your hotel.
* **Mid-morning:** Visit the Panathenaic Stadium, where the first modern Olympic Games were held. (~20-minute walk from Syntagma Square or a short metro ride, free entry)
* **Lunch:** Have a farewell lunch at **Oinomageireio Epirus** in Syntagma Square, a traditional taverna serving authentic Greek dishes.
* **Afternoon:** Depart from Athens International Airport (ATH).

### Transportation:

* **Metro:** Athens has an efficient metro system that connects most major attractions. You can purchase tickets at stations or online.
* **Taxis:** Taxis are readily available throughout the city and can be hailed on the street or pre-booked through **Athens Taxi Wagon** or **Taxicity**.
* **Walking:** Many attractions are within walking distance of each other, especially in the Plaka and Monastiraki areas.

This itinerary prioritizes your preferences for delicious food, comfortable accommodations, and enjoying a cocktail or two. You'll have ample opportunities to indulge in culinary delights, relax in cozy beds, and sip on refreshing cocktails while experiencing the many wonders of Athens. Remember to adjust the restaurant choices based on your specific dietary preferences and budget. Enjoy your trip!

In [ ]:
prompt5 = 'please give me the itinerary again with the costs added too!'
response5 = get_chat_response(chat, prompt5)
display(Markdown(response5))


**Day 1**

* **Morning:** Check in to your hotel, **The Artist Athens** (approx. €120/night).
* **Afternoon:** Visit the **Acropolis Museum** (approx. €10 entrance fee; 45 min. walk). Explore the museum's collection of ancient artifacts from the Acropolis site.
* **Dinner:** Have a traditional Greek meal at **Psomi & Alati** near the Acropolis (approx. €20; 15 min. walk).

**Day 2**

* **Morning:** Take a **guided tour of the Acropolis** with **Chat Tours** (approx. €30; 2 hours). Learn about the history and significance of this iconic site.
* **Afternoon:** Stroll through the picturesque neighborhood of **Plaka** (15 min. walk). Visit boutiques and admire the charming architecture.
* **Dinner:** Dine at **Mani Mani** in Plaka for authentic Greek cuisine (approx. €25; 20 min. walk).

**Day 3**

* **Morning:** Visit the **National Archaeological Museum** (approx. €10 entrance fee; 20 min. walk). Discover a vast collection of Greek antiquities, including sculptures, pottery, and jewelry.
* **Afternoon:** Take an **Athens by Bike** guided tour (approx. €35; 3 hours; book online). Explore the city's highlights from a unique perspective.
* **Dinner:** Try modern Greek cuisine at **Thes "Greek Creative Cuisine"** (approx. €25; 25 min. walk).

**Day 4**

* **Morning:** Take a day trip to the island of **Hydra** by ferry (approx. €25 round-trip; 1.5 hours by ferry). Explore the car-free streets, visit art galleries, and enjoy the scenic harbor.
* **Afternoon:** Return to Athens in the evening.
* **Dinner:** Have a farewell meal at **Belpaese** (approx. €25; 15 min. walk).

**Day 5**

* **Morning:** Visit the **Museum of Cycladic Art** (approx. €10 entrance fee; 30 min. walk). Admire the extensive collection of ancient Cycladic sculptures and artifacts.
* **Afternoon:** Visit the **Benaki Museum** (approx. €10 entrance fee; 20 min. walk). Explore its collections of Greek and Islamic art.
* **Dinner:** Enjoy a rooftop meal with panoramic city views at **V1935 Roof Garden** near Syntagma Square (approx. €30; 15 min. walk).

**Day 6**

* **Morning:** Check out of your hotel.
* **Afternoon:** Explore the vibrant flea market in **Monastiraki** (approx. 15 min. walk). Browse for unique souvenirs and antiques.
* **Evening:** Depart from Athens.

**Total Cost:** Approx. €340 (excluding breakfast and any additional expenses).

**Note:** The walking distances and times provided are approximate and may vary depending on your pace and the specific route you take.

Above we can see the output after constructing a series of 3 prompts that improves on the base prompt. Few-shot prompting allows the LLM to complete multiple tasks when putting them all in a single prompt is too much for it too handle.

**Saving the selected itinerary and its components:**

It also may be useful save the list of trip components that the LLM chose. For example we may to save this to a user's profile, pull up a few reviews for the user, or even show the user a map with the locations of their trip components.

We could ask the LLM to format these and feed them back into a structure that we can parse. But this is an expensive operation. Below we do introduce a function that takes the prompt output, and matches exact strings to get a list of components that we incorporated.


In [29]:
###First we remember which components we included in the first prompt
hotel_components = prompt_output[1]
attraction_components = prompt_output[2]
restaurant_components = prompt_output[3]

In [30]:
##Takes a list of components, and a text, and returns a list of components that were found in the text
def chosen_components(components, text):
    chosen = []
    for component in components:
        if component in text:
            chosen.append(component)

    return chosen

In [31]:
def get_URLs(chosen, df):  ##chosen is a list of component names, df is the master database csv.
    URLs = ' \n '
    for i in chosen:
        URLs+= '\n ' + i + ': ' + df[df['name'] == i]['webUrl'].values[0] + '\n '
    return URLs

In [32]:
def create_links_list(restaurant_components, hotel_components, attraction_components, response):
  restaurant_components = chosen_components(restaurant_components, response)
  hotel_components = chosen_components(hotel_components, response)
  attraction_components = chosen_components(attraction_components, response)

  ##Takes in the components originally used to construct the initial prompt, and the response from the LLM
  ##Returns a list of TripAdvisor links for those components
  all_components = get_URLs(restaurant_components, df_restaurants_c) + get_URLs(hotel_components, df_hotels_c) + get_URLs(attraction_components, df_attractions_c)
  return all_components

In [33]:
response_with_links = response4 + create_links_list(restaurant_components, hotel_components, attraction_components, response3)

In [34]:
display(Markdown(response_with_links))

## Optimized 7-Night Athens Itinerary for Foodies: Prioritizing Food, Cozy Beds, and Cocktails

This revised itinerary prioritizes your love for delicious food, comfortable accommodations, and enjoying a cocktail or two. It ensures you have cozy beds to sink into after a day of exploring and indulging in culinary delights.

### Day 1:

* **Morning:** Arrive at Athens International Airport (ATH). Take the metro line 3 (blue line) directly to Syntagma Square (~40 minutes, €10). Check into your chosen hotel:
    * **Luxury:** Hotel Grande Bretagne, A Luxury Collection Hotel (5-star, centrally located, stunning views, €350+/night, luxurious beds, elegant rooftop bar)
    * **Mid-range:** O&B Athens Boutique Hotel (4-star, stylish, rooftop terrace, €150+/night, comfortable beds, rooftop bar with city views)
* **Lunch:** Enjoy a traditional Greek lunch at **Ta Karamanlidika tou Fani** in Plaka (5-minute walk from Syntagma Square, €20-30 per person, delicious meze platters, friendly atmosphere).
* **Afternoon:** Explore the Acropolis complex, including the Parthenon, Erechtheion, and Temple of Athena Nike. Join a guided tour with **Nectar & Ambrosia** for in-depth insights and skip-the-line access. (~2-hour tour, €70 per person)
* **Evening:** Dine at **Oroscopo**, a Michelin-starred restaurant with innovative Greek cuisine and stunning Acropolis views. Afterward, head to the rooftop bar **A for Athens** for cocktails with panoramic city vistas. (Both within a 5-minute walk of the Acropolis, €50-70 per person for dinner, €15-20 per cocktail)

### Day 2:

* **Morning:** Visit the National Archaeological Museum, home to an extensive collection of ancient Greek artifacts. (~15-minute walk from Plaka or a short metro ride, €10 entrance fee)
* **Lunch:** Have a casual lunch at **Smile Restaurant**, offering traditional Greek cuisine near the museum (~5-minute walk, €15-20 per person, delicious homemade dishes, friendly service).
* **Afternoon:** Stroll through the charming Plaka district, with its narrow streets, souvenir shops, and historical buildings.
* **Evening:** Experience a unique dining experience at **Dinner in the Sky**, enjoying a gourmet meal suspended 50 meters above the ground with breathtaking views. (Location varies, transportation time may vary, €300 per person)
* **Later:** Relax and enjoy a cocktail at the **Brettos Bar** in Monastiraki, renowned for its live music and energetic atmosphere.

### Day 3:

* **Morning:** Take a scenic hike up Mount Lycabettus, offering panoramic views of the city. (~15-minute walk from Syntagma Square or a short taxi ride, €5-10 taxi fare)
* **Lunch:** Have a picnic lunch at the top of Mount Lycabettus, enjoying the breathtaking vistas.
* **Afternoon:** Learn to cook traditional Greek dishes at a hands-on cooking class with **Cookly** in Monastiraki. (~15-minute walk from Mount Lycabettus or a short metro ride, €70 per person)
* **Evening:** Enjoy the fruits of your labor at the cooking class itself, savoring the dishes you prepared.
* **Later:** Experience the vibrant nightlife of Athens at **Brettos Bar** in Monastiraki, renowned for its live music and energetic atmosphere.

### Day 4:

* **Morning:** Visit the Benaki Museum, housing a diverse collection of Greek art and history. (~10-minute walk from Monastiraki or a short metro ride, €10 entrance fee)
* **Lunch:** Have a delightful lunch at **Funky Gourmet** in Neos Kosmos, a Michelin-starred restaurant with creative and playful interpretations of Greek cuisine (~30-minute metro ride from Monastiraki, €50-70 per person, creative dishes, innovative cocktails).
* **Afternoon:** Relax and rejuvenate at a traditional hammam, such as the **Hammam Baths of Athens** in Monastiraki. (~10-minute walk from the Benaki Museum, €40 per person)
* **Evening:** Dine at **Botrini's** in Kifissia, a contemporary fine dining restaurant with a focus on seasonal ingredients. (~40-minute taxi ride from Monastiraki, €100+ per person, exquisite cuisine, sophisticated ambiance).

### Day 5:

* **Morning:** Take a full-day cruise to the Saronic Islands, including Hydra, Poros, and Aegina. Relax on the beaches, explore the picturesque villages, and enjoy swimming in the crystal-clear waters. (~1-hour ferry ride from Piraeus port, €50-70 per person)
* **Lunch:** Enjoy a delicious seafood lunch onboard the cruise.
* **Dinner:** Have a casual dinner at a taverna on one of the islands, savoring the local cuisine and ambiance.

### Day 6:

* **Morning:** Visit the Acropolis Museum, showcasing the archaeological finds from the Acropolis. (~15-minute walk from Plaka or a short metro ride, €10 entrance fee)
* **Lunch:** Have a traditional Greek lunch at **Oroscopo** in Plaka, a Michelin-starred restaurant with innovative Greek cuisine and stunning Acropolis views.
* **Afternoon:** Explore the vibrant Monastiraki flea market, searching for unique souvenirs and hidden treasures.
* **Evening:** Spend a memorable evening at the **Gazarte**, a cultural center with live music, art exhibitions, and a variety of restaurants and bars. (10-minute walk from Monastiraki)

### Day 7:

* **Morning:** Enjoy a leisurely breakfast at your hotel.
* **Mid-morning:** Visit the Panathenaic Stadium, where the first modern Olympic Games were held. (~20-minute walk from Syntagma Square or a short metro ride, free entry)
* **Lunch:** Have a farewell lunch at **Oinomageireio Epirus** in Syntagma Square, a traditional taverna serving authentic Greek dishes.
* **Afternoon:** Depart from Athens International Airport (ATH).

### Transportation:

* **Metro:** Athens has an efficient metro system that connects most major attractions. You can purchase tickets at stations or online.
* **Taxis:** Taxis are readily available throughout the city and can be hailed on the street or pre-booked through **Athens Taxi Wagon** or **Taxicity**.
* **Walking:** Many attractions are within walking distance of each other, especially in the Plaka and Monastiraki areas.

This itinerary prioritizes your preferences for delicious food, comfortable accommodations, and enjoying a cocktail or two. You'll have ample opportunities to indulge in culinary delights, relax in cozy beds, and sip on refreshing cocktails while experiencing the many wonders of Athens. Remember to adjust the restaurant choices based on your specific dietary preferences and budget. Enjoy your trip! 
 
 Lithos: https://www.tripadvisor.com/Restaurant_Review-g189400-d1151377-Reviews-Lithos-Athens_Attica.html
 
 Ta Karamanlidika tou Fani: https://www.tripadvisor.com/Restaurant_Review-g189400-d6852056-Reviews-Ta_Karamanlidika_tou_Fani-Athens_Attica.html
 
 Oroscopo: https://www.tripadvisor.com/Restaurant_Review-g189400-d2186282-Reviews-Oroscopo-Athens_Attica.html
 
 Smile Restaurant: https://www.tripadvisor.com/Restaurant_Review-g189400-d1902225-Reviews-Smile_Restaurant-Athens_Attica.html
 
 Oinomageireio Epirus: https://www.tripadvisor.com/Restaurant_Review-g189400-d7794299-Reviews-Oinomageireio_Epirus-Athens_Attica.html
 
 Dinner in the Sky: https://www.tripadvisor.com/Restaurant_Review-g189400-d7825999-Reviews-Dinner_in_the_Sky-Athens_Attica.html
  
 
 Hotel Grande Bretagne, A Luxury Collection Hotel: https://www.tripadvisor.com/Hotel_Review-g189400-d198715-Reviews-Hotel_Grande_Bretagne_A_Luxury_Collection_Hotel-Athens_Attica.html
 
 O&B Athens Boutique Hotel: https://www.tripadvisor.com/Hotel_Review-g189400-d607425-Reviews-O_B_Athens_Boutique_Hotel-Athens_Attica.html
  
 
 Acropolis Museum: https://www.tripadvisor.com/Attraction_Review-g189400-d735521-Reviews-Acropolis_Museum-Athens_Attica.html
 
 Plaka: https://www.tripadvisor.com/Attraction_Review-g189400-d277449-Reviews-Plaka-Athens_Attica.html
 
 Acropolis: https://www.tripadvisor.com/Attraction_Review-g189400-d198706-Reviews-Acropolis-Athens_Attica.html
 
 National Archaeological Museum: https://www.tripadvisor.com/Attraction_Review-g189400-d198713-Reviews-National_Archaeological_Museum-Athens_Attica.html
 
 Monastiraki: https://www.tripadvisor.com/Attraction_Review-g189400-d265865-Reviews-Monastiraki-Athens_Attica.html
 
 Parthenon: https://www.tripadvisor.com/Attraction_Review-g189400-d198711-Reviews-Parthenon-Athens_Attica.html
 
 Brettos Bar: https://www.tripadvisor.com/Attraction_Review-g189400-d523153-Reviews-Brettos_Bar-Athens_Attica.html
 
 Athens Taxi Wagon: https://www.tripadvisor.com/Attraction_Review-g189400-d12232077-Reviews-Athens_Taxi_Wagon-Athens_Attica.html
 
 Taxicity: https://www.tripadvisor.com/Attraction_Review-g189400-d7276748-Reviews-Taxicity-Athens_Attica.html
 
 Panathenaic Stadium: https://www.tripadvisor.com/Attraction_Review-g189400-d245991-Reviews-Panathenaic_Stadium-Athens_Attica.html
 
 Mount Lycabettus: https://www.tripadvisor.com/Attraction_Review-g189400-d245989-Reviews-Mount_Lycabettus-Athens_Attica.html
 